In [1]:
import numpy as np
import pandas as pd
import shutil
from roboflow import Roboflow
import os
import re

In [2]:
def clean_up(integral=False):
    if os.path.exists('GARBAGE-CLASSIFICATION-3-2'):
        shutil.rmtree('GARBAGE-CLASSIFICATION-3-2')
    if integral:
        if os.path.exists('data'):
            shutil.rmtree('data')
    else:
        for file in os.listdir('data'):
            file = os.path.join('data', file)
            if os.path.isfile(file):
                os.remove(file)

def move_files_recursively(src,dest='data',files=None):
    if not os.path.exists(dest):
        os.makedirs(dest)
    files_to_move = []
    if files is None:
        for root, dirs, files in os.walk(src):
            for file in files:
                if file.endswith('.jpg'):
                    files_to_move.append(os.path.join(root, file))
    else:
        interesting_files = list(set(files) & set(os.listdir(src)))
        for file in interesting_files:
            files_to_move.append(os.path.join(src, file))
    for file in files_to_move:
        shutil.move(file, dest)

In [3]:
clean_up(True)
rf = Roboflow(api_key="PYZngL70ijcagLdXpb6l")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in tensorflow:: 100%|██████████| 10472/10472 [00:00<00:00, 11910.31it/s]


In [4]:
train = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/train/_annotations.csv')
test = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/test/_annotations.csv')
val = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/valid/_annotations.csv')
combined_data = pd.concat([train, test, val], ignore_index=True)

In [5]:
move_files_recursively('GARBAGE-CLASSIFICATION-3-2')

In [6]:
map_classes = {
    'METAL': 'METAL_CARDBOARD_PLASTIC',
    'CARDBOARD': 'METAL_CARDBOARD_PLASTIC',
    'PLASTIC': 'METAL_CARDBOARD_PLASTIC',
    'PAPER': 'PAPER',
    'GLASS': 'GLASS',
    'BIODEGRADABLE': 'BIODEGRADABLE'
}
def get_class(c):
    for key in map_classes.keys():
        if key==c:
            return map_classes.get(key)
    return c

def remove_until_char(input_string,remove_char,include_char=True):
    match = re.search(remove_char, input_string)
    if match:
        return input_string[match.start() + 1 if include_char else 0:]
    return input_string

def get_set_name(ratios=None):
    if ratios is None:
        ratios = [0.6, 0.25, 0.15]
    rand = np.random.uniform()
    if rand <= ratios[0]:
        set_name = 'TRAIN'
    elif rand <= ratios[0] + ratios[1]:
        set_name = 'TEST'
    else:
        set_name = 'VAL'
    return set_name

def edit_row(row):
    row['class'] = get_class(row['class'])
    return row

for index, row in combined_data.iterrows():
    combined_data.loc[index] = edit_row(row)

classes_occurences = combined_data['class'].value_counts()    
classes_occurences

class
BIODEGRADABLE              45407
METAL_CARDBOARD_PLASTIC    16484
GLASS                       7809
PAPER                       4390
Name: count, dtype: int64

In [15]:
ratios = [0.6, 0.25, 0.15]
cleansed_data = pd.DataFrame(columns=['set','filename','label','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4'])
for index, row in combined_data.iterrows():
    width = row['width']
    height = row['height']
    row['label']=row['class']
    row['xmin_relative'] = row['xmin'] / width
    row['ymin_relative'] = row['ymin'] / height
    row['xmax_relative'] = row['xmax'] / width
    row['ymax_relative'] = row['ymax'] / height
    row['set'] = get_set_name(ratios)
    row['1'] = None
    row['2'] = None
    row['3'] = None
    row['4'] = None
    cleansed_data.loc[index] = row[['set','filename','label','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4']]
    if index % 500 == 0:
        print(f'Processed {index+1}/{combined_data.shape[0]}')
cleansed_data.to_csv('annotations_full.csv', index=False)

Processed 1/74090
Processed 501/74090
Processed 1001/74090
Processed 1501/74090


KeyboardInterrupt: 

## Balancing data 

In [29]:
cleansed_data = pd.read_csv('annotations_full.csv')
classes_occurences = cleansed_data['label'].value_counts()
classes_occurences

label
BIODEGRADABLE              45407
METAL_CARDBOARD_PLASTIC    16484
GLASS                       7809
PAPER                       4390
Name: count, dtype: int64

In [33]:
balanced_data = pd.DataFrame(columns=cleansed_data.columns)
for class_label in cleansed_data['label'].unique():
    class_samples = cleansed_data[cleansed_data['label'] == class_label].head(80)#np.min(classes_occurences))
    balanced_data = pd.concat([balanced_data, class_samples], ignore_index=True)
print(f'Balanced dataframe shape: {balanced_data.shape}')
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
print(f'Balanced dataframe shape shuffled: {balanced_data.shape}')

Balanced dataframe shape: (320, 11)
Balanced dataframe shape shuffled: (320, 11)


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [34]:
balanced_data.to_csv('data/annotations.csv', index=False)
balanced_data

,set,filename,label,xmin_relative,ymin_relative,1,2,xmax_relative,ymax_relative,3,4
0,TRAIN,plastic731_jpg.rf.0944892364ec72286d65188b0fb8...,METAL_CARDBOARD_PLASTIC,0.045673,0.817308,NaN,NaN,0.973558,0.980769,NaN,NaN
1,TRAIN,glass2883_jpg.rf.0a0547f44666a5045f40079d04ec0...,GLASS,0.137019,0.120192,NaN,NaN,0.899038,0.262019,NaN,NaN
2,TRAIN,plastic229_jpg.rf.097a45689640e8fae5e67e67e09e...,METAL_CARDBOARD_PLASTIC,0.552885,0.314904,NaN,NaN,0.788462,0.747596,NaN,NaN
3,TRAIN,glass2314_jpg.rf.09d2982348e38de382dac53bc7464...,GLASS,0.127404,0.002404,NaN,NaN,0.831731,0.918269,NaN,NaN
4,TEST,glass1911_jpg.rf.09cf543ca5e97d469cb8246edee3e...,GLASS,0.086538,0.774038,NaN,NaN,0.615385,0.995192,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
315,TRAIN,glass1911_jpg.rf.09cf543ca5e97d469cb8246edee3e...,GLASS,0.055288,0.225962,NaN,NaN,0.574519,0.432692,NaN,NaN
316,VAL,paper405_jpg.rf.10c397807456491d318739f2ca3c5d...,PAPER,0.000000,0.000000,NaN,NaN,0.781250,0.348558,NaN,NaN
317,TRAIN,paper1661_jpeg.rf.0b1512f4228353998c63e9c22d97...,PAPER,0.401442,0.423077,NaN,NaN,0.670673,0.807692,NaN,NaN
318,TRAIN,paper1569_jpeg.rf.09320ab28d28db478fd550133513...,PAPER,0.064904,0.168269,NaN,NaN,0.987981,0.793269,NaN,NaN
